In [1]:
# Import Libraries
import pandas as pd
import xlwings as xw
from xlwings import constants

In [2]:
# Import DataFrame
df = pd.read_csv('FakeData.csv')

In [3]:
# Create Workbook
wb = xw.Book()
sheet = wb.sheets['Sheet1']

In [4]:
# Add dates to spreadsheet
for x in range(len(df['DateNow'].unique())):
    sheet.cells(2, (x+3)).value = df['DateNow'].unique()[x]

In [5]:
# Unique categories in each section
completed_cat = df.loc[df.FlagStatus == 1]['Categories'].unique()
afterhours_cat = df.loc[df.AfterHours == 1.0]['Categories'].unique()
carriedover_cat = df.loc[df.CarriedOver == 1.0]['Categories'].unique()
pastsla_cat = df.loc[df.PastSLA == 1.0]['Categories'].unique()

# Calculate category lengths
completed_cat_length = len(completed_cat)
afterhours_cat_length = len(afterhours_cat)
carriedover_cat_length = len(carriedover_cat)
pastsla_cat_length = len(pastsla_cat)

In [6]:
# Add Completed categories to spreadsheet
for x in range(completed_cat_length):
    sheet.cells((3+x), 2).value = completed_cat[x]
    
# Add After Hours categories to spreadsheet
for x in range(afterhours_cat_length):
    sheet.cells((4 + completed_cat_length + x), 2).value = afterhours_cat[x]
    
# Add Carried Over categories to spreadsheet
for x in range(carriedover_cat_length):
    sheet.cells((5 + completed_cat_length + afterhours_cat_length + x), 2).value = carriedover_cat[x]

# Add Past SLA categories to spreadsheet
for x in range(pastsla_cat_length):
    sheet.cells((6 + completed_cat_length + afterhours_cat_length + carriedover_cat_length + x), 2).value = pastsla_cat[x]

In [7]:
# Add daily numbers by category and status
for d in range(len(df['DateNow'].unique())):
    
    # Completed
    for key, value in dict(df.loc[(df.DateNow == df.DateNow.unique()[d]) & (df.FlagStatus == 1)].Categories.value_counts()).items():
        for x in range(completed_cat_length):
            for key, value in dict(df.loc[(df.DateNow == df.DateNow.unique()[d]) & (df.FlagStatus == 1)].Categories.value_counts()).items():
                if key == sheet.cells((3 + x), 2).value:
                    sheet.cells((3 + x), (3 + d)).value = value
    
    # After Hours
    for key, value in dict(df.loc[(df.DateNow == df.DateNow.unique()[d]) & (df.FlagStatus != 1) & (df.AfterHours == 1.0)].Categories.value_counts()).items():
        for x in range(completed_cat_length):
            for key, value in dict(df.loc[(df.DateNow == df.DateNow.unique()[d]) & (df.FlagStatus != 1) & (df.AfterHours == 1.0)].Categories.value_counts()).items():
                if key == sheet.cells((4 + x + completed_cat_length), 2).value:
                    sheet.cells((4 + completed_cat_length + x), (3 + d)).value = value
                    
    # Carried Over
    for key, value in dict(df.loc[(df.DateNow == df.DateNow.unique()[d]) & (df.FlagStatus != 1) & (df.CarriedOver == 1.0)].Categories.value_counts()).items():
        for x in range(completed_cat_length):
            for key, value in dict(df.loc[(df.DateNow == df.DateNow.unique()[d]) & (df.FlagStatus != 1) & (df.CarriedOver == 1.0)].Categories.value_counts()).items():
                if key == sheet.cells((5 + x + completed_cat_length + afterhours_cat_length), 2).value:
                    sheet.cells((5 + completed_cat_length + afterhours_cat_length + x), (3 + d)).value = value

    # Past SLA
    for key, value in dict(df.loc[(df.DateNow == df.DateNow.unique()[d]) & (df.FlagStatus != 1) & (df.PastSLA == 1.0)].Categories.value_counts()).items():
        for x in range(completed_cat_length):
            for key, value in dict(df.loc[(df.DateNow == df.DateNow.unique()[d]) & (df.FlagStatus != 1) & (df.PastSLA == 1.0)].Categories.value_counts()).items():
                if key == sheet.cells((6 + x + completed_cat_length + afterhours_cat_length + carriedover_cat_length), 2).value:
                    sheet.cells((6 + completed_cat_length + afterhours_cat_length + carriedover_cat_length + x), (3 + d)).value = value

In [8]:
# Add daily status totals
for d in range(len(df['DateNow'].unique())):
    sheet.cells((3 + completed_cat_length), (3 + d)).value = len(df.loc[(df.DateNow == df.DateNow.unique()[d]) & (df.FlagStatus == 1)])
    
    sheet.cells((4 + completed_cat_length + afterhours_cat_length), (3 + d)).value = len(df.loc[(df.DateNow == df.DateNow.unique()[d]) & (df.FlagStatus != 1) & (df.AfterHours == 1.0)])
    
    sheet.cells((5 + completed_cat_length + afterhours_cat_length + carriedover_cat_length), (3 + d)).value = len(df.loc[(df.DateNow == df.DateNow.unique()[d]) & (df.FlagStatus != 1) & (df.CarriedOver == 1.0)])
    
    sheet.cells((6 + completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length), (3 + d)).value = len(df.loc[(df.DateNow == df.DateNow.unique()[d]) & (df.FlagStatus != 1) & (df.PastSLA == 1.0)])

In [9]:
# Total header
sheet.cells(2, (len(df['DateNow'].unique())) + 3).value = 'Total'

# Average header
sheet.cells(2, (len(df['DateNow'].unique())) + 4).value = 'Average'

# Percent Header
sheet.cells(2, (len(df['DateNow'].unique())) + 5).value = 'Percent'

In [10]:
# Calculate monthly totals
for c in range(completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 4):
    cattotal = 0
    for x in range(len(df['DateNow'].unique())):
        if sheet.cells(c + 3, x + 3).value != None:
            cattotal = cattotal + sheet.cells(c + 3, x + 3).value
        else:
            pass
    sheet.cells((c + 3), (len(df['DateNow'].unique()) + 3)).value = cattotal
    
# Calculate monthly averages
for c in range(completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 4):
    cataverage = sheet.cells((c + 3), (len(df['DateNow'].unique()) + 3)).value / (len(df['DateNow'].unique()))
    sheet.cells((c + 3), (len(df['DateNow'].unique()) + 4)).value = cataverage
    

In [11]:
# Add total percent of box by category

# Completed
for c in range(completed_cat_length):
    sheet.cells((c + 3), (len(df['DateNow'].unique()) + 5)).value = (sheet.cells((c + 3), (len(df['DateNow'].unique()) + 3)).value / sheet.cells((completed_cat_length + 3), (len(df['DateNow'].unique()) + 3)).value)*100
    
# After Hours
for c in range(afterhours_cat_length):
    sheet.cells((c + completed_cat_length + 4), (len(df['DateNow'].unique()) + 5)).value = (sheet.cells((c + completed_cat_length + 4), (len(df['DateNow'].unique()) + 3)).value / sheet.cells((afterhours_cat_length + completed_cat_length + 4), (len(df['DateNow'].unique()) + 3)).value)*100
    
# Carried Over
for c in range(carriedover_cat_length):
    sheet.cells((c + completed_cat_length + afterhours_cat_length + 5), (len(df['DateNow'].unique()) + 5)).value = (sheet.cells((c + carriedover_cat_length + completed_cat_length + 5), (len(df['DateNow'].unique()) + 3)).value / sheet.cells((carriedover_cat_length + afterhours_cat_length + completed_cat_length + 5), (len(df['DateNow'].unique()) + 3)).value)*100
    
# Past SLA
for c in range(pastsla_cat_length):
    sheet.cells((c + completed_cat_length + carriedover_cat_length + afterhours_cat_length + 6), (len(df['DateNow'].unique()) + 5)).value = (sheet.cells((c + carriedover_cat_length + completed_cat_length + pastsla_cat_length + 6), (len(df['DateNow'].unique()) + 3)).value / sheet.cells((carriedover_cat_length + afterhours_cat_length + completed_cat_length + pastsla_cat_length + 6), (len(df['DateNow'].unique()) + 3)).value)*100

In [12]:
# Add status headers
sheet.cells(3, 1).value = "Completed"
sheet.cells((completed_cat_length + 4), 1).value = "After Hours\nreceived after 16:30, one business day ago"
sheet.cells((completed_cat_length + afterhours_cat_length + 5), 1).value = "Carried Over\nreceived after 16:30, two business days ago"
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 1).value = "Past SLA\nreceived prior to 16:30, two business days ago"

In [13]:
# Merge status headers

# Completed
sheet.range(sheet.cells(3, 1), sheet.cells((completed_cat_length + 3), 1)).api.MergeCells = True

# After Hours
sheet.range(sheet.cells((completed_cat_length + 4), 1), sheet.cells((completed_cat_length + afterhours_cat_length + 4), 1)).api.MergeCells = True

# Carried Over
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 5), 1), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 5), 1)).api.MergeCells = True

# Past SLA
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 1), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6), 1)).api.MergeCells = True

In [14]:
# Add date header
sheet.cells(1, 3).value = "Date"

In [15]:
# Merge date header
sheet.range(sheet.cells(1, 3), sheet.cells(1, (len(df['DateNow'].unique())) + 2)).api.MergeCells = True

In [16]:
# Merge cleanup
sheet.range(sheet.cells(1, 1), sheet.cells(2, 2)).api.MergeCells = True

In [17]:
# Horizontal and vertical alignment for status headers

# Completed
sheet.cells(3, 1).api.HorizontalAlignment = constants.HAlign.xlHAlignCenter
sheet.cells(3, 1).api.VerticalAlignment = constants.VAlign.xlVAlignCenter

# After Hours
sheet.cells((completed_cat_length + 4), 1).api.HorizontalAlignment = constants.HAlign.xlHAlignCenter
sheet.cells((completed_cat_length + 4), 1).api.VerticalAlignment = constants.VAlign.xlVAlignCenter

# Carried Over
sheet.cells((completed_cat_length + afterhours_cat_length + 5), 1).api.HorizontalAlignment = constants.HAlign.xlHAlignCenter
sheet.cells((completed_cat_length + afterhours_cat_length + 5), 1).api.VerticalAlignment = constants.VAlign.xlVAlignCenter

# Past SLA
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 1).api.HorizontalAlignment = constants.HAlign.xlHAlignCenter
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 1).api.VerticalAlignment = constants.VAlign.xlVAlignCenter

In [18]:
# Horizontal and vertical alignment for date headers
sheet.cells(1, 3).api.HorizontalAlignment = constants.HAlign.xlHAlignCenter
sheet.cells(1, 3).api.VerticalAlignment = constants.VAlign.xlVAlignCenter

In [19]:
# Convert rgb to integer
def rgbToInt(rgb):
    colorInt = rgb[0] + (rgb[1] * 256) + (rgb[2] * 256 * 256)
    return colorInt

In [20]:
# Cell formatting

# Completed header
sheet.cells(3,1).api.GetCharacters(1,9).Font.Bold = True
sheet.cells(3,1).api.GetCharacters(1,9).Font.Size = 14
sheet.cells(3,1).api.Interior.Color = rgbToInt((195,221,161))
sheet.cells(3,1).api.Font.Color = rgbToInt((49,49,45))

# After Hours header
sheet.cells((completed_cat_length + 4), 1).api.GetCharacters(1,11).Font.Bold = True
sheet.cells((completed_cat_length + 4), 1).api.GetCharacters(1,11).Font.Size = 14
sheet.cells((completed_cat_length + 4), 1).api.Interior.Color = rgbToInt((185,211,236))
sheet.cells((completed_cat_length + 4), 1).api.Font.Color = rgbToInt((49,49,45))
sheet.cells((completed_cat_length + 4), 1).api.GetCharacters(12,50).Font.Size = 8

# Carried Over header
sheet.cells((completed_cat_length + afterhours_cat_length + 5), 1).api.GetCharacters(1,12).Font.Bold = True
sheet.cells((completed_cat_length + afterhours_cat_length + 5), 1).api.GetCharacters(1,12).Font.Size = 14
sheet.cells((completed_cat_length + afterhours_cat_length + 5), 1).api.Interior.Color = rgbToInt((207,171,227))
sheet.cells((completed_cat_length + afterhours_cat_length + 5), 1).api.Font.Color = rgbToInt((49,49,45))
sheet.cells((completed_cat_length + afterhours_cat_length + 5), 1).api.GetCharacters(13,50).Font.Size = 8

# Past SLA header
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 1).api.GetCharacters(1,8).Font.Bold = True
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 1).api.GetCharacters(1,8).Font.Size = 14
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 1).api.Interior.Color = rgbToInt((244,116,107))
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 1).api.Font.Color = rgbToInt((49,49,45))
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 1).api.GetCharacters(9,50).Font.Size = 8

# Date header
sheet.cells(1,3).api.Font.Bold = True
sheet.cells(1,3).api.Font.Size = 14
sheet.cells(1,3).api.Interior.Color = rgbToInt((0,0,0))
sheet.cells(1,3).api.Font.Color = rgbToInt((246,246,244))
sheet.cells(1,1).api.Interior.Color = rgbToInt((0,0,0))
sheet.cells(1,(len(df['DateNow'].unique()) + 3)).api.Interior.Color = rgbToInt((0,0,0))
sheet.cells(1,(len(df['DateNow'].unique()) + 4)).api.Interior.Color = rgbToInt((0,0,0))
sheet.cells(1,(len(df['DateNow'].unique()) + 5)).api.Interior.Color = rgbToInt((0,0,0))

# Date
sheet.range(sheet.cells(2, 3), sheet.cells(2,(len(df['DateNow'].unique()) + 2))).api.Interior.Color = rgbToInt((49,49,45))
sheet.range(sheet.cells(2, 3), sheet.cells(2,(len(df['DateNow'].unique()) + 2))).api.Font.Color = rgbToInt((246,246,244))

# Total header
sheet.cells(2,(len(df['DateNow'].unique()) + 3)).api.Interior.Color = rgbToInt((0,0,0))
sheet.cells(2,(len(df['DateNow'].unique()) + 3)).api.Font.Color = rgbToInt((246,246,244))

# Average header
sheet.cells(2,(len(df['DateNow'].unique()) + 4)).api.Interior.Color = rgbToInt((49,49,45))
sheet.cells(2,(len(df['DateNow'].unique()) + 4)).api.Font.Color = rgbToInt((246,246,244))

# Percent header
sheet.cells(2,(len(df['DateNow'].unique()) + 5)).api.Interior.Color = rgbToInt((49,49,45))
sheet.cells(2,(len(df['DateNow'].unique()) + 5)).api.Font.Color = rgbToInt((246,246,244))

# Completed categories
sheet.range(sheet.cells(3, 2), sheet.cells((completed_cat_length + 2), 2)).api.Interior.Color = rgbToInt((215,233,193))
sheet.range(sheet.cells(3, 2), sheet.cells((completed_cat_length + 2), 2)).api.Font.Color = rgbToInt((49,49,45))

# After Hours categories
sheet.range(sheet.cells((completed_cat_length + 4), 2), sheet.cells((completed_cat_length + afterhours_cat_length + 3), 2)).api.Interior.Color = rgbToInt((209,226,241))
sheet.range(sheet.cells((completed_cat_length + 4), 2), sheet.cells((completed_cat_length + afterhours_cat_length + 3), 2)).api.Font.Color = rgbToInt((49,49,45))

# Carried Over categories
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 5), 2), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 4), 2)).api.Interior.Color = rgbToInt((226,206,238))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 5), 2), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 4), 2)).api.Font.Color = rgbToInt((49,49,45))

# Past SLA categories
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 2), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 5), 2)).api.Interior.Color = rgbToInt((249,168,155))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 2), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 5), 2)).api.Font.Color = rgbToInt((49,49,45))

# Completed daily numbers
sheet.range(sheet.cells(3, 3), sheet.cells((completed_cat_length + 2), (len(df['DateNow'].unique()) + 2))).api.Interior.Color = rgbToInt((235,244,224))
sheet.range(sheet.cells(3, 3), sheet.cells((completed_cat_length + 2), (len(df['DateNow'].unique()) + 2))).api.Font.Color = rgbToInt((49,49,45))

# After Hours daily numbers
sheet.range(sheet.cells((completed_cat_length + 4), 3), sheet.cells((completed_cat_length + afterhours_cat_length + 3), (len(df['DateNow'].unique()) + 2))).api.Interior.Color = rgbToInt((231,240,249))
sheet.range(sheet.cells((completed_cat_length + 4), 3), sheet.cells((completed_cat_length + afterhours_cat_length + 3), (len(df['DateNow'].unique()) + 2))).api.Font.Color = rgbToInt((49,49,45))

# Carried Over daily numbers
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 5), 3), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 4), (len(df['DateNow'].unique()) + 2))).api.Interior.Color = rgbToInt((247,240,250))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 5), 3), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 4), (len(df['DateNow'].unique()) + 2))).api.Font.Color = rgbToInt((49,49,45))

# Past SLA daily numbers
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 3), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 5), (len(df['DateNow'].unique()) + 2))).api.Interior.Color = rgbToInt((254,206,193))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), 3), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 5), (len(df['DateNow'].unique()) + 2))).api.Font.Color = rgbToInt((49,49,45))

# Completed daily total
sheet.range(sheet.cells((completed_cat_length + 3), 2), sheet.cells((completed_cat_length + 3), (len(df['DateNow'].unique()) + 5))).api.Interior.Color = rgbToInt((135,187,77))
sheet.range(sheet.cells((completed_cat_length + 3), 2), sheet.cells((completed_cat_length + 3), (len(df['DateNow'].unique()) + 5))).api.Font.Color = rgbToInt((246,246,244))

# After Hours daily total
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 4), 2), sheet.cells((completed_cat_length + afterhours_cat_length + 4), (len(df['DateNow'].unique()) + 5))).api.Interior.Color = rgbToInt((99,175,216))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 4), 2), sheet.cells((completed_cat_length + afterhours_cat_length + 4), (len(df['DateNow'].unique()) + 5))).api.Font.Color = rgbToInt((246,246,244))

# Carried Over daily total
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 5), 2), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 5), (len(df['DateNow'].unique()) + 5))).api.Interior.Color = rgbToInt((187,119,215))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 5), 2), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 5), (len(df['DateNow'].unique()) + 5))).api.Font.Color = rgbToInt((246,246,244))

# Past SLA daily total
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6), 2), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6), (len(df['DateNow'].unique()) + 5))).api.Interior.Color = rgbToInt((224,15,14))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6), 2), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6), (len(df['DateNow'].unique()) + 5))).api.Font.Color = rgbToInt((246,246,244))

# Total - Completed total
sheet.range(sheet.cells(3, (len(df['DateNow'].unique()) + 3)), sheet.cells((completed_cat_length + 2), (len(df['DateNow'].unique()) + 3))).api.Interior.Color = rgbToInt((49,49,45))
sheet.range(sheet.cells(3, (len(df['DateNow'].unique()) + 3)), sheet.cells((completed_cat_length + 2), (len(df['DateNow'].unique()) + 3))).api.Font.Color = rgbToInt((246,246,244))

# Average - Completed total
sheet.range(sheet.cells(3, (len(df['DateNow'].unique()) + 4)), sheet.cells((completed_cat_length + 2), (len(df['DateNow'].unique()) + 4))).api.Interior.Color = rgbToInt((82,81,74))
sheet.range(sheet.cells(3, (len(df['DateNow'].unique()) + 4)), sheet.cells((completed_cat_length + 2), (len(df['DateNow'].unique()) + 4))).api.Font.Color = rgbToInt((246,246,244))

# Percent - Completed total
sheet.range(sheet.cells(3, (len(df['DateNow'].unique()) + 5)), sheet.cells((completed_cat_length + 2), (len(df['DateNow'].unique()) + 5))).api.Interior.Color = rgbToInt((82,81,74))
sheet.range(sheet.cells(3, (len(df['DateNow'].unique()) + 5)), sheet.cells((completed_cat_length + 2), (len(df['DateNow'].unique()) + 5))).api.Font.Color = rgbToInt((246,246,244))

# Total - After Hours total
sheet.range(sheet.cells((completed_cat_length + 4), (len(df['DateNow'].unique()) + 3)), sheet.cells((completed_cat_length + afterhours_cat_length + 3), (len(df['DateNow'].unique()) + 3))).api.Interior.Color = rgbToInt((49,49,45))
sheet.range(sheet.cells((completed_cat_length + 4), (len(df['DateNow'].unique()) + 3)), sheet.cells((completed_cat_length + afterhours_cat_length + 3), (len(df['DateNow'].unique()) + 3))).api.Font.Color = rgbToInt((246,246,244))

# Average - After Hours total
sheet.range(sheet.cells((completed_cat_length + 4), (len(df['DateNow'].unique()) + 4)), sheet.cells((completed_cat_length + afterhours_cat_length + 3), (len(df['DateNow'].unique()) + 4))).api.Interior.Color = rgbToInt((82,81,74))
sheet.range(sheet.cells((completed_cat_length + 4), (len(df['DateNow'].unique()) + 4)), sheet.cells((completed_cat_length + afterhours_cat_length + 3), (len(df['DateNow'].unique()) + 4))).api.Font.Color = rgbToInt((246,246,244))

# Percent - After Hours total
sheet.range(sheet.cells((completed_cat_length + 4), (len(df['DateNow'].unique()) + 5)), sheet.cells((completed_cat_length + afterhours_cat_length + 3), (len(df['DateNow'].unique()) + 5))).api.Interior.Color = rgbToInt((82,81,74))
sheet.range(sheet.cells((completed_cat_length + 4), (len(df['DateNow'].unique()) + 5)), sheet.cells((completed_cat_length + afterhours_cat_length + 3), (len(df['DateNow'].unique()) + 5))).api.Font.Color = rgbToInt((246,246,244))

# Total - Carried Over total
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 5), (len(df['DateNow'].unique()) + 3)), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 4), (len(df['DateNow'].unique()) + 3))).api.Interior.Color = rgbToInt((49,49,45))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 5), (len(df['DateNow'].unique()) + 3)), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 4), (len(df['DateNow'].unique()) + 3))).api.Font.Color = rgbToInt((246,246,244))

# Average - Carried Over total
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 5), (len(df['DateNow'].unique()) + 4)), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 4), (len(df['DateNow'].unique()) + 4))).api.Interior.Color = rgbToInt((82,81,74))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 5), (len(df['DateNow'].unique()) + 4)), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 4), (len(df['DateNow'].unique()) + 4))).api.Font.Color = rgbToInt((246,246,244))

# Percent - Carried Over total
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 5), (len(df['DateNow'].unique()) + 5)), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 4), (len(df['DateNow'].unique()) + 5))).api.Interior.Color = rgbToInt((82,81,74))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 5), (len(df['DateNow'].unique()) + 5)), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 4), (len(df['DateNow'].unique()) + 5))).api.Font.Color = rgbToInt((246,246,244))

# Total - Past SLA total
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), (len(df['DateNow'].unique()) + 3)), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 5), (len(df['DateNow'].unique()) + 3))).api.Interior.Color = rgbToInt((49,49,45))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), (len(df['DateNow'].unique()) + 3)), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 5), (len(df['DateNow'].unique()) + 3))).api.Font.Color = rgbToInt((246,246,244))

# Average - Past SLA total
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), (len(df['DateNow'].unique()) + 4)), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 5), (len(df['DateNow'].unique()) + 4))).api.Interior.Color = rgbToInt((82,81,74))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), (len(df['DateNow'].unique()) + 4)), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 5), (len(df['DateNow'].unique()) + 4))).api.Font.Color = rgbToInt((246,246,244))

# Percent - Past SLA total
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), (len(df['DateNow'].unique()) + 5)), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 5), (len(df['DateNow'].unique()) + 5))).api.Interior.Color = rgbToInt((82,81,74))
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 6), (len(df['DateNow'].unique()) + 5)), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 5), (len(df['DateNow'].unique()) + 5))).api.Font.Color = rgbToInt((246,246,244))

# Completed total totals
sheet.cells((completed_cat_length + 3), (len(df['DateNow'].unique()) + 3)).api.Interior.Color = rgbToInt((68,101,39))
sheet.cells((completed_cat_length + 3), (len(df['DateNow'].unique()) + 4)).api.Interior.Color = rgbToInt((78,118,45))
sheet.cells((completed_cat_length + 3), (len(df['DateNow'].unique()) + 5)).api.Interior.Color = rgbToInt((78,118,45))

# After Hours total totals
sheet.cells((completed_cat_length + afterhours_cat_length + 4), (len(df['DateNow'].unique()) + 3)).api.Interior.Color = rgbToInt((40,98,140))
sheet.cells((completed_cat_length + afterhours_cat_length + 4), (len(df['DateNow'].unique()) + 4)).api.Interior.Color = rgbToInt((52,126,170))
sheet.cells((completed_cat_length + afterhours_cat_length + 4), (len(df['DateNow'].unique()) + 5)).api.Interior.Color = rgbToInt((52,126,170))

# Carried Over total totals
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 5), (len(df['DateNow'].unique()) + 3)).api.Interior.Color = rgbToInt((152,64,183))
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 5), (len(df['DateNow'].unique()) + 4)).api.Interior.Color = rgbToInt((174,79,208))
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 5), (len(df['DateNow'].unique()) + 5)).api.Interior.Color = rgbToInt((174,79,208))

# Past SLA total totals
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6), (len(df['DateNow'].unique()) + 3)).api.Interior.Color = rgbToInt((193,22,1))
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6), (len(df['DateNow'].unique()) + 4)).api.Interior.Color = rgbToInt((207,24,1))
sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6), (len(df['DateNow'].unique()) + 5)).api.Interior.Color = rgbToInt((207,24,1))


In [21]:
# Add row borders by status

# Completed
sheet.range(sheet.cells((completed_cat_length + 3), 1), sheet.cells((completed_cat_length + 3),(len(df['DateNow'].unique()) + 5))).api.Borders(9).LineStyle = 1
sheet.range(sheet.cells((completed_cat_length + 3), 1), sheet.cells((completed_cat_length + 3),(len(df['DateNow'].unique()) + 5))).api.Borders(9).Weight = 2

# After Hours
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 4), 1), sheet.cells((completed_cat_length + afterhours_cat_length + 4),(len(df['DateNow'].unique()) + 5))).api.Borders(9).LineStyle = 1
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + 4), 1), sheet.cells((completed_cat_length + afterhours_cat_length + 4),(len(df['DateNow'].unique()) + 5))).api.Borders(9).Weight = 2

# Carried Over
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 5), 1), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 5),(len(df['DateNow'].unique()) + 5))).api.Borders(9).LineStyle = 1
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 5), 1), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + 5),(len(df['DateNow'].unique()) + 5))).api.Borders(9).Weight = 2

# Past SLA
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6), 1), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6),(len(df['DateNow'].unique()) + 5))).api.Borders(9).LineStyle = 1
sheet.range(sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6), 1), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6),(len(df['DateNow'].unique()) + 5))).api.Borders(9).Weight = 2

In [22]:
# Autofit rows and columns
sheet.range(sheet.cells(1,1), sheet.cells((completed_cat_length + afterhours_cat_length + carriedover_cat_length + pastsla_cat_length + 6), (len(df['DateNow'].unique()) + 5))).autofit()

In [23]:
sheet.name = 'NACustomerMaintenance'

In [24]:
# Save workbook
wb.save('FakeTracker.xlsx')